In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [4]:
# Configurations
input_directory = r"C:\Users\33658\Documents\CelebrityFacesCropped\Celebrity Faces Cropped"  # Dataset cropped
image_size = (224, 224)  # Taille des images pour MobileNetV2
learning_rate = 0.0008
batch_size = 32


In [5]:

# Étape 1 : Préparer les données
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normaliser les pixels
    validation_split=0.2,  # 20% pour la validation
    horizontal_flip=True,  # Augmenter les données
    zoom_range=0.2
)

train_generator = datagen.flow_from_directory(
    input_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

validation_generator = datagen.flow_from_directory(
    input_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

Found 565 images belonging to 9 classes.
Found 136 images belonging to 9 classes.


In [6]:
# Étape 2 : Construire le modèle
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Geler les couches du modèle de base
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Étape 3 : Compiler le modèle
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss="categorical_crossentropy",
              metrics=["accuracy"])


In [8]:
from tensorflow.keras.callbacks import EarlyStopping

# Configurer Early Stopping
early_stopping = EarlyStopping(
    monitor="val_loss",  # Surveiller la perte sur l'ensemble de validation
    patience=25,          # Nombre d'époques sans amélioration avant d'arrêter
    restore_best_weights=True,  # Restaurer les poids correspondant à la meilleure époque
    verbose=1            # Afficher un message lorsque l'entraînement s'arrête
)


In [9]:
# Étape 4 : Entraîner le modèle
num_epochs = 125

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=num_epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping]  # Ajouter le callback Early Stopping
)

Epoch 1/125


C:\Users\33658\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 819ms/step - accuracy: 0.2254 - loss: 2.1532 - val_accuracy: 0.5234 - val_loss: 1.4384
Epoch 2/125
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3750 - loss: 1.5860 - val_accuracy: 0.7500 - val_loss: 1.5546
Epoch 3/125


C:\Users\33658\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 395ms/step - accuracy: 0.5897 - loss: 1.3766 - val_accuracy: 0.7031 - val_loss: 1.0577
Epoch 4/125
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6875 - loss: 1.1964 - val_accuracy: 0.6250 - val_loss: 1.4525
Epoch 5/125
17/17 ━━━━━━━━━━━━━━━━━━━━ 9s 393ms/step - accuracy: 0.6684 - loss: 1.0371 - val_accuracy: 0.7969 - val_loss: 0.8049
Epoch 6/125
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7188 - loss: 0.9673 - val_accuracy: 0.8750 - val_loss: 0.5283
Epoch 7/125
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 401ms/step - accuracy: 0.6830 - loss: 0.9719 - val_accuracy: 0.8203 - val_loss: 0.7080
Epoch 8/125
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8750 - loss: 0.5542 - val_accuracy: 0.5000 - val_loss: 1.4103
Epoch 9/125
17/17 ━━━━━━━━━━━━━━━━━━━━ 9s 422ms/step - accuracy: 0.7652 - loss: 0.7587 - val_accuracy: 0.8438 - val_loss: 0.6565
Epoch 10/125
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8438 - loss: 0.5845 - val_accuracy: 0.8750 - val_

In [10]:
# Étape 5 : Sauvegarder le modèle
model.save("actor_recognition_model_night.h5")
model.save('actor_recognition_model_night.keras')
print("Modèle sauvegardé avec succès !")

Modèle sauvegardé avec succès !


Test du modele

In [10]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np

# Charger le modèle
model = load_model("/content/drive/MyDrive/Colab Notebooks/actor_recognition_model_night.h5")

# Charger une image de test
img_path = "/content/drive/MyDrive/Colab Notebooks/Data/cruise_test2.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Prédiction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
class_labels = list(train_generator.class_indices.keys())
print(f"L'acteur prédit est : {class_labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
L'acteur prédit est : Brad Pitt


In [12]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Charger le modèle
model = load_model("/content/drive/MyDrive/Colab Notebooks/actor_recognition_model_night.h5")

# Recompiler le modèle (si nécessaire)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Configurer les chemins
train_directory = "/content/drive/MyDrive/Colab Notebooks/Data/Celebrity Faces Cropped"  # Dossier utilisé pour l'entraînement
test_directory = "/content/drive/MyDrive/Colab Notebooks/Data/Celebrity Faces Cropped/Tom Cruise"  # Dossier de test
image_size = (224, 224)  # Taille d'entrée des images

# Récupérer les noms des classes depuis les sous-dossiers
datagen = ImageDataGenerator(rescale=1.0 / 255)
generator = datagen.flow_from_directory(
    train_directory,
    target_size=image_size,
    batch_size=32,
    class_mode="categorical"
)
class_labels = list(generator.class_indices.keys())  # Liste des noms de classes
print(f"Classes détectées : {class_labels}")

# Fonction pour prédire une image
def predict_image(image_path, model):
    # Charger et prétraiter l'image
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Faire une prédiction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = predictions[0][predicted_class]

    return class_labels[predicted_class], confidence

# Parcourir les images du dossier et faire des prédictions
for img_name in os.listdir(test_directory):
    img_path = os.path.join(test_directory, img_name)

    # Vérifier que le fichier est une image
    if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
        predicted_actor, confidence = predict_image(img_path, model)
        print(f"Image: {img_name} => Acteur prédit: {predicted_actor} (Confiance: {confidence:.2f})")
    else:
        print(f"Fichier ignoré : {img_name}")


Found 701 images belonging to 9 classes.
Classes détectées : ['Brad Pitt', 'Hugh Jackman', 'Johnny Depp', 'Leonardo DiCaprio', 'Natalie Portman', 'Robert Downey Jr', 'Tom Cruise', 'Tom Hanks', 'Will Smith']
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Image: 036_f47a2dc9.jpg => Acteur prédit: Tom Cruise (Confiance: 0.47)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Image: 048_21c4f84a.jpg => Acteur prédit: Tom Cruise (Confiance: 0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Image: 003_ed3fb7b1.jpg => Acteur prédit: Tom Cruise (Confiance: 0.96)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Image: 013_712a38d6.jpg => Acteur prédit: Tom Cruise (Confiance: 0.55)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Image: 057_e3da0420.jpg => Acteur prédit: Tom Cruise (Confiance: 0.90)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Image: 083_63bb9285.jpg => Acteur prédit: Tom Cruise (Confiance: 0.88)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Image: 059_6b515be7.jpg => Acteur prédit: Tom Cruise (Confiance: 0.77)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

In [21]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Charger le modèle
model = load_model("actor_recognition_model_night.h5")

# Recompiler le modèle (si nécessaire)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Configurer les chemins
train_directory = r"C:\Users\33658\Documents\CelebrityFacesCropped\Celebrity Faces Cropped"  # Dossier utilisé pour l'entraînement
test_directory = r"C:\Users\33658\Documents\CelebrityFacesCropped\Celebrity Faces Cropped\Hugh Jackman"  # Dossier de test
image_size = (224, 224)  # Taille d'entrée des images

# Récupérer les noms des classes depuis les sous-dossiers
datagen = ImageDataGenerator(rescale=1.0 / 255)
generator = datagen.flow_from_directory(
    train_directory,
    target_size=image_size,
    batch_size=32,
    class_mode="categorical"
)
class_labels = list(generator.class_indices.keys())  # Liste des noms de classes
print(f"Classes détectées : {class_labels}")

# Fonction pour prédire une image
def predict_image(image_path, model):
    # Charger et prétraiter l'image
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Faire une prédiction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = predictions[0][predicted_class]

    return predicted_class, confidence

# Évaluer les performances globales
correct_predictions = 0
total_images = 0

# Parcourir les images du dossier et faire des prédictions
for img_name in os.listdir(test_directory):
    img_path = os.path.join(test_directory, img_name)

    # Vérifier que le fichier est une image
    if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
        predicted_class, confidence = predict_image(img_path, model)
        true_class = class_labels.index("Hugh Jackman")  # Remplacer par la classe correcte pour ce dossier

        if predicted_class == true_class:
            correct_predictions += 1
        total_images += 1

        print(f"Image: {img_name} => Acteur prédit: {class_labels[predicted_class]} (Confiance: {confidence:.2f})")
    else:
        print(f"Fichier ignoré : {img_name}")

# Calculer et afficher la performance globale
accuracy = (correct_predictions / total_images) * 100 if total_images > 0 else 0
print(f"Total d'images évaluées : {total_images}")
print(f"Prédictions correctes : {correct_predictions}")
print(f"Précision globale : {accuracy:.2f}%")


Found 701 images belonging to 9 classes.
Classes détectées : ['Brad Pitt', 'Hugh Jackman', 'Johnny Depp', 'Leonardo DiCaprio', 'Natalie Portman', 'Robert Downey Jr', 'Tom Cruise', 'Tom Hanks', 'Will Smith']
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 727ms/step
Image: 001_9adc92c2.jpg => Acteur prédit: Hugh Jackman (Confiance: 0.64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: 002_85eab275.jpg => Acteur prédit: Hugh Jackman (Confiance: 0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: 003_8889ec2c.jpg => Acteur prédit: Hugh Jackman (Confiance: 0.97)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image: 004_41caa173.jpg => Acteur prédit: Hugh Jackman (Confiance: 0.59)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image: 005_3ba56da0.jpg => Acteur prédit: Hugh Jackman (Confiance: 0.86)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: 006_ff6876d9.jpg => Acteur prédit: Hugh Jackman (Confiance: 0.95)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: 007_68abd54d.jpg => Acteur prédit: Hugh Jackman (Confiance: 0.97)
1/1 ━━━━━━━━━━━